# 로지스틱 회귀

In [1]:
import pandas as pd

df = pd.read_csv('data/Fish.csv')

In [2]:
df['Species'].unique()

array(['Bream', 'Roach', 'Whitefish', 'Parkki', 'Perch', 'Pike', 'Smelt'],
      dtype=object)

In [3]:
fish_input = df[['Weight', 'Length2', 'Length3', 'Height', 'Width']] # 종을 빼고 나머지 열을 입력데이터로 사용
# fish_input.head()

In [4]:
#### 다중분류 = 타깃 데이터에 2개 이상의 클래스가 포함된 문제

In [5]:
fish_target = df[['Species']] 

In [6]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = \
train_test_split(fish_input, fish_target)

In [7]:
# 훈련세트와 테스트 세트를 표준화 전처리하기

In [8]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

In [9]:
##### k-최근접 이웃 분류기의 확률 예측
    # - 여러개의 클래스들이 있으면 주변의 클래스들의 확률을 계산해 가장 높은 결과가 나오게

In [10]:

from sklearn.neighbors import KNeighborsClassifier

kn = KNeighborsClassifier()
kn.fit(train_scaled, train_target)

print(kn.score(train_scaled, train_target))
print(kn.score(test_scaled, test_target)) 

0.8823529411764706
0.675


C:\Users\yerin\Desktop\DAMF\ML\venv\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [11]:
print(kn.classes_)

['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']


In [12]:
# 다른 특성들로 종을 예측했을 때 나오는 결과들
kn.predict(test_scaled)

array(['Perch', 'Perch', 'Smelt', 'Pike', 'Perch', 'Perch', 'Perch',
       'Roach', 'Parkki', 'Perch', 'Pike', 'Smelt', 'Perch', 'Bream',
       'Bream', 'Perch', 'Perch', 'Perch', 'Perch', 'Perch', 'Perch',
       'Perch', 'Perch', 'Perch', 'Pike', 'Perch', 'Bream', 'Smelt',
       'Smelt', 'Perch', 'Perch', 'Pike', 'Perch', 'Perch', 'Pike',
       'Bream', 'Pike', 'Perch', 'Perch', 'Perch'], dtype=object)

In [13]:
# 클래스별 확률값 반환 predict_proba()
# 첫줄의 0은 'bream'의 확률, 0.6 = 'parkki' 0 = 'perch', 0.2 = 'pike' ... 
# 이웃한 샘플의 클래스 비율이므로 항상 정해진 확률만 출력 
kn.predict_proba(test_scaled)

array([[0. , 0. , 0.6, 0. , 0.4, 0. , 0. ],
       [0. , 0. , 0.6, 0. , 0.4, 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 1. , 0. ],
       [0. , 0. , 0.4, 0.6, 0. , 0. , 0. ],
       [0. , 0. , 0.8, 0. , 0.2, 0. , 0. ],
       [0. , 0. , 0.6, 0. , 0.2, 0.2, 0. ],
       [0.2, 0. , 0.6, 0. , 0.2, 0. , 0. ],
       [0. , 0. , 0.4, 0. , 0.6, 0. , 0. ],
       [0. , 0.8, 0.2, 0. , 0. , 0. , 0. ],
       [0. , 0. , 1. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 1. , 0. , 0. , 0. ],
       [0. , 0. , 0.2, 0. , 0.2, 0.6, 0. ],
       [0. , 0. , 0.8, 0. , 0.2, 0. , 0. ],
       [1. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0.6, 0.2, 0. , 0. , 0.2, 0. , 0. ],
       [0. , 0. , 0.8, 0. , 0.2, 0. , 0. ],
       [0. , 0. , 1. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 1. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 1. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0.6, 0. , 0.4, 0. , 0. ],
       [0. , 0. , 0.8, 0. , 0. , 0. , 0.2],
       [0. , 0. , 1. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 1. , 0. , 0. ,

##### 로지스틱 회귀 : z 값을 도출 (회귀) ,  z 값으로 확률을 도출 (분류)
    - 선형 회귀와 동일하게 선형 방정식을 학습
       - ex) z = a * Weight + b * Length2 + c * Length3 + d * Height + e * Width + f
       - a,b,c,d,e는 가중치 혹은 계수
       - z는 어떤 값도 가능하지만, 확률이 되려면 0 ~ 1 (또는 0 ~ 100%) 사이 값이 되어야 함
       - z가 아주 큰 음수일 때 0이 되고, z가 아주 큰 양수일 때 1이 되도록 바꿔야 함 > 시그모이드 함수(로지스틱 함수) 이용
       - 이진 분류에서는 하나의 선형 방정식을 훈련, 이 방정식의 출력값을 시그모이드 함수에 통과시켜 0 ~ 1 사이의 값을 만듦
       - 다중 분류일 경우에는 클래스 개수만큼 방정식을 훈련, 그 다음 각 방정식의 출력값을 소프트맥스 함수를 통과시켜 전체 클래스           에 대한 합이 항상 1이 되도록 만듦

In [14]:
train_scaled
train_target

,Species
109,Perch
86,Perch
72,Perch
34,Bream
102,Perch
...,...
121,Perch
98,Perch
76,Perch
141,Pike


In [15]:
# 훈련 세트에서 도미와 빙어의 행만 골라내기 (이진 분류)
# 비교 연산자를 사용하면 도미와 빙어의 행을 모두 True로 만들기
# 
bream_smelt_indexes = (train_target == 'Bream') | (train_target == 'Smelt')
# bream_smelt_indexes['Species'] = 도미와 빙어일 경우 True이고 그 외는 모두 False 값이 들어가 있음
train_bs = train_scaled[bream_smelt_indexes['Species']]
target_bs = train_target[bream_smelt_indexes['Species']]
target_bs

,Species
34,Bream
17,Bream
148,Smelt
3,Bream
150,Smelt
31,Bream
6,Bream
149,Smelt
152,Smelt
16,Bream


In [16]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(train_bs, target_bs)

C:\Users\yerin\Desktop\DAMF\ML\venv\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [17]:
import numpy as np
print(lr.predict(train_bs[:5])) # 5개의 샘플 예측해보기
lr.predict_proba(train_bs[:5]).round(3) # 예측 확률 보기 

['Bream' 'Bream' 'Smelt' 'Bream' 'Smelt']


array([[0.999, 0.001],
       [0.994, 0.006],
       [0.037, 0.963],
       [0.967, 0.033],
       [0.035, 0.965]])

In [18]:
print(lr.coef_, lr.intercept_) 
# z = -0.420 * Weight + -0.610 * Length2 + -0.692 * Length3 + -0.976 * Height + -0.748 * Width + -2.539

[[-0.43250142 -0.63849546 -0.71954575 -1.0151514  -0.80835693]] [-2.50277434]


In [19]:
# 로지스틱 회귀로 다중 분류 수행하기 (7종의 생선 분류해보기)

In [20]:
lr = LogisticRegression(C=20, max_iter=1000) # max_iter = 반복 횟수를 지정, 기본값은 100
# c = 계수의 제곱을 규제, 작을수록 규제가 커짐, 기본값은 1
lr.fit(train_scaled, train_target) # 7마리의 생선 

print(lr.score(train_scaled, train_target))
print(lr.score(test_scaled, test_target))

0.957983193277311
0.8


C:\Users\yerin\Desktop\DAMF\ML\venv\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [21]:
lr.predict(test_scaled[:5])

array(['Roach', 'Perch', 'Smelt', 'Pike', 'Perch'], dtype=object)

In [22]:
lr.predict_proba(test_scaled[:5]).round(3) 
# 5개의 샘플에 대한 예측으로 5개의 행이 출력, 7개의 생선에 대한 확률 계산으로 7개의 열 출력

array([[0.002, 0.009, 0.365, 0.003, 0.616, 0.   , 0.005],
       [0.   , 0.003, 0.764, 0.005, 0.226, 0.   , 0.001],
       [0.   , 0.007, 0.04 , 0.   , 0.012, 0.942, 0.   ],
       [0.   , 0.   , 0.003, 0.997, 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.888, 0.   , 0.11 , 0.   , 0.   ]])

In [23]:
lr.coef_.shape # 5개의 특성을 사용하고 z를 7개 계산한다는 의미

(7, 5)

In [24]:
# 다중 분류는 클래스마다 z 값을 하나씩 계산, 당연히 가장 높은 z값을 출력하는 클래스가 예측 클래스가 됨
# 이진 분류에서는 시그모이드 함수를 사용해 z를 0과 1 사이의 값으로 변환
# 다중 분류는 이와 달리 소프트맥스 함수를 사용하여 7개의 z값을 확률로 변환